# Elección y puesta en marcha de modelo de ML

Vamos a cargar nuevamente los datos para trabajar en este jupyternotebook.

In [1]:
import pandas as pd

#Cargamos los datos
datos=pd.read_excel("E-Commerce_train.xlsx")

#Separamos nuestra variable objetivo
objetivo=datos["Reached.on.Time_Y.N"]

Ahora aplicamos una secuencia de ColumnTransformer para la transformación de nuestros datos así como los seleccionamos en el [análisis exploratorio de datos](EDA.ipynb)

In [3]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler

pipeline_completo = ColumnTransformer([
        ("descuento", 
        MinMaxScaler(feature_range=(.01,.65), copy=False), 
        ['Discount_offered']),
        ("precio", 
        MinMaxScaler(feature_range=(1,10), copy=False), 
        ['Cost_of_the_Product']),
    ])

variables = pipeline_completo.fit_transform(datos)

Verificamos que todo haya salido según lo planeado

In [6]:
variables

array([[0.44      , 4.40654206],
       [0.59      , 6.04672897],
       [0.48      , 4.6588785 ],
       ...,
       [0.06      , 7.89719626],
       [0.05      , 4.70093458],
       [0.04      , 3.06074766]])

In [7]:
variables.shape

(8999, 2)

Podemos ver que ahora sí tenemos nuestros datos listos para producción. Lo único que faltaría hacer es separar nuestros datos en un conjunto de prueba y un conjunto de entrenamiento

In [8]:
from sklearn.model_selection import train_test_split

v_entreno, v_prueba, o_entreno, o_prueba = train_test_split(variables, objetivo, test_size=0.30, random_state=9)

Tratandose de un problema sencillo de regreción vamos a iniciar utilizando una regreción logística. En todos nuestros modelos los vamos a entrenar con una semilla de 9 a modo de poder mantener los resultados a lo largo de las ejecuciones.

In [32]:
from sklearn.linear_model import LogisticRegression

#instanciamos el modelo
modeloRegresion = LogisticRegression(random_state=9)

#entrenamos el modelo
modeloRegresion.fit(v_entreno,o_entreno)

LogisticRegression(random_state=9)

Como lo que nos interesa es mejorar el accuracy y el recall de nuestras funciones, vamos a crear una función que los evalúe y nos lo muestre, tanto para los datos de entrenamiento y los datos de prueba

In [33]:
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
def puntajes(modelo, o_entreno, o_prueba, v_entreno, v_prueba):
    
    prueba_predicho = modelo.predict(v_prueba)
    entreno_predicho = modelo.predict(v_entreno)

    # Nos aseguramos que la entrada sea en forma de integer 
    # para que lo soporten las distintas funciones
    recal=recall_score(o_prueba, prueba_predicho.astype(int))
    acur= accuracy_score(o_prueba, prueba_predicho.astype(int))
    recale=recall_score(o_entreno, entreno_predicho.astype(int))
    acure= accuracy_score(o_entreno, entreno_predicho.astype(int))
    print("Recall de prueba: ", recal)
    print("Accuracy de prueba: ", acur)
    print("Recall de entreno: ", recale)
    print("Accuracy de entreno: ", acure)

In [34]:
puntajes(modeloRegresion, o_entreno, o_prueba, v_entreno, v_prueba)

Recall de prueba:  0.7753537735849056
Accuracy de prueba:  0.6403703703703704
Recall de entreno:  0.7651006711409396
Accuracy de entreno:  0.6404191141451024


Vamos a intentar ahora con un arbol binario de clasificación

In [27]:
from sklearn.tree import DecisionTreeRegressor

modelo_arbol = DecisionTreeRegressor(random_state=9)

modelo_arbol.fit(v_entreno,o_entreno)

DecisionTreeRegressor(random_state=9)

In [35]:
puntajes(modelo_arbol, o_entreno, o_prueba, v_entreno, v_prueba)

Recall de prueba:  0.5778301886792453
Accuracy de prueba:  0.6562962962962963
Recall de entreno:  0.6164553815560527
Accuracy de entreno:  0.7550404826162883


Vamos a intentar ahora con un modelo de clasificación SVC

In [38]:
from sklearn.svm import SVC

modeloSVC = SVC(random_state=9)

modeloSVC.fit(v_entreno,o_entreno)

SVC(random_state=9)

In [39]:
puntajes(modeloSVC, o_entreno, o_prueba, v_entreno, v_prueba)

Recall de prueba:  1.0
Accuracy de prueba:  0.6281481481481481
Recall de entreno:  1.0
Accuracy de entreno:  0.6386728052071757


Vemos que este modelo puntua muy bien en cuanto a nuestra variable objetivo y tiene un accuracy bastante bueno. Sin embargo vamos a ver si un random Forest Clasifier puede superarlo

In [42]:
from sklearn.ensemble import RandomForestClassifier

modeloForest = RandomForestClassifier(random_state=9)

modeloForest = modeloForest.fit(v_entreno, o_entreno)

In [43]:
puntajes(modeloForest, o_entreno, o_prueba, v_entreno, v_prueba)

Recall de prueba:  0.7352594339622641
Accuracy de prueba:  0.6548148148148148
Recall de entreno:  0.8535918468804374
Accuracy de entreno:  0.8171138275916813


Como nuestro objetivo es obtener un buen Recall nos vamos a quedar con el modelo SVC y vamos a intentar conseguir los mejores hiperparámetros para intentar mejorar su accuracy

In [57]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import make_scorer
from sklearn.metrics import recall_score

"""scoring = { 
            "Recall": make_scorer(recall_score), 
            "Accuracy": make_scorer(accuracy_score)
          }"""

parametros={
            "kernel":["sigmoid", "rbf"],
            "coef0":[.0,.3,.6,.9],
            "gamma":["scale", "auto"],
            }

modeloSVC = SVC(random_state=9)

grid_search = GridSearchCV(modeloSVC, parametros, cv=5,
                           scoring=make_scorer(accuracy_score),
                           return_train_score=True,
                           refit="accuracy",
                           n_jobs=-1)

grid_search.fit(v_entreno, o_entreno)

GridSearchCV(cv=5, estimator=SVC(random_state=9), n_jobs=-1,
             param_grid={'coef0': [0.0, 0.3, 0.6, 0.9],
                         'gamma': ['scale', 'auto'],
                         'kernel': ['sigmoid', 'rbf']},
             refit='accuracy', return_train_score=True,
             scoring=make_scorer(accuracy_score))

In [58]:
grid_search.best_params_

{'coef0': 0.0, 'gamma': 'scale', 'kernel': 'rbf'}

Vemos cuales son los mejores hiperparámetros para nuestro modelo nos damos cuenta que son los que se usa por defecto en las predicciones, aún así los escribimos

In [67]:
modeloSVC = SVC(kernel='rbf', gamma='scale', coef0=0.0,random_state=9)

#Entrenamos el modelo
modeloSVC.fit(v_entreno,o_entreno)

#analizamos su puntaje
puntajes(modeloSVC, o_entreno, o_prueba, v_entreno, v_prueba)

Recall de prueba:  1.0
Accuracy de prueba:  0.6281481481481481
Recall de entreno:  1.0
Accuracy de entreno:  0.6386728052071757


Comprobamos que definitivamente es identico al que calculamos anteriormente. De todas formas lo seleccionamos para enviar

In [68]:
#Cargamos los datos
datos_deploy=pd.read_excel("E-Commerce_test.xlsx")

In [69]:
variables_deploy = pipeline_completo.fit_transform(datos_deploy)

In [72]:
salida = modeloSVC.predict(variables_deploy)

In [73]:
salida=pd.DataFrame(salida,columns=["pred"])

In [74]:
salida.to_csv("sypten.csv",index=False)